In [1]:
import numpy as np
import pandas as pd
from os import path as osp
import os
from matplotlib import pyplot as plt
import lightgbm as lgb

import pickle
def save_obj(obj, name):
    with open(name + '.pickle', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open(name + '.pickle', 'rb') as f:
        return pickle.load(f)

In [2]:
DATA_PATH = '/data'
MODEL_PAHT = '/work/model'
SEED = 2222
if not os.path.exists(MODEL_PAHT):
    os.mkdir(MODEL_PAHT)
    
data_type='train'

In [3]:
train_base = pd.read_csv(osp.join(DATA_PATH, f'{data_type}_base.csv'))
train_base = train_base.sort_values(['cust_wid']).reset_index(drop=True)
train_view = pd.read_csv(osp.join(DATA_PATH, f'{data_type}_view.csv'), encoding='gbk')
train_trx = pd.read_csv(osp.join(DATA_PATH, f'{data_type}_trx.csv'))

# deal base gdr_cd
train_base['gdr_cd'] = train_base['gdr_cd'].map({'M':1, 'F':2})
train_base['gdr_cd'] = train_base['gdr_cd'].fillna(0)

# deal base cty_cd
train_base['cty_cd'] = train_base['cty_cd'].str.slice(stop=2)
train_base.cty_cd = train_base.cty_cd.fillna('NAN')
cty_map = ['C', '44', '11', 'A', '31', '32', '37', '21', '42', '61', 'B', '33', '50', '12', '35', '36', '51', '43', '62', '53', '34', '23', '65', '41', 'NAN']
cty_map = dict(zip(cty_map, range(1,len(cty_map)+1)))
train_base['cty_cd'] = train_base['cty_cd'].map(cty_map).fillna(0)

# idx = train_view.acs_tm.str.contains('下午')
train_view.acs_tm = train_view.acs_tm.str.rstrip(':').str.replace('上午|下午', '', regex=True).str.replace('1492', '2022')
train_view.acs_tm = pd.to_datetime(train_view.acs_tm)
train_view['day'] = train_view.acs_tm.dt.day

train_trx.trx_tm = train_trx.trx_tm.str.replace('1492', '2022')
train_trx.trx_tm = pd.to_datetime(train_trx.trx_tm)
train_trx['day'] = train_trx.trx_tm.dt.day

if data_type=='train':
    train_view['day'] -= 4
    train_trx['day'] -= 4
elif data_type=='testa':
    train_view['day'] -= 3
    train_trx['day'] -= 3

# # train_view['day'] -= 14
# # train_trx['day'] -= 14

# print(train_trx['day'].max(), train_trx['day'].min())

# # train_view = train_view.query('day>=0').reset_index(drop=True)
# # train_trx = train_trx.query('day>=0').reset_index(drop=True)

# train_view['day'] %= 14
# train_trx['day'] %= 14
# train_view['day'] = train_view['day'].astype('int')
# train_trx['day'] = train_trx['day'].astype('int')

# train_view = train_view.sort_values(['cust_wid','acs_tm']).reset_index(drop=True)
# train_trx = train_trx.sort_values(['cust_wid','trx_tm']).reset_index(drop=True)


In [4]:
idx = train_view['day'].isna()

In [5]:
train_trx = train_trx.merge(train_base[["cust_wid", "label"]], on='cust_wid')
train_trx['label'] = (train_trx['label']>0).astype('int')
trx_target_enc = train_trx.groupby('trx_cd')['label'].mean().to_dict()
save_obj(trx_target_enc, osp.join(MODEL_PAHT, 'trx_id_enc'))

In [6]:
train_view['page_id'].value_counts().shape

(2326,)

In [7]:
page = train_view['page_id'].value_counts()
page = page[page>40].index.to_list()

train_view = train_view.merge(train_base[["cust_wid", "label"]], on='cust_wid')
idx = train_view.page_id.isin(page)
train_view.loc[~idx, 'page_id'] = 'NAN'
train_view['label'] = (train_view['label']>0).astype('int')
print('NAN', train_view.page_id.value_counts()['NAN'])
page_target_enc = train_view.groupby('page_id')['label'].mean().to_dict()
save_obj(page_target_enc, osp.join(MODEL_PAHT, 'page_id_enc'))

NAN 16711


In [ ]:
len(page)

In [9]:
# 2326
# 10: 6928  1542
# 20: 10366 1313
# 40: 16711 1097
# 50: 20215 1020